<a href="https://colab.research.google.com/github/danielelbrecht/CAP5610_HW3/blob/master/CP5610_HW3_Problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.applications import VGG16
from keras import backend as K
from keras import layers
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, utils

# Import VGG model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


58892288/58889256 [==============================] - 5s 0us/step


# Fine tuning with data augmentation

In [4]:
# Download cats and dogs data set
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2019-04-18 01:56:44--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  80.6MB/s    in 0.8s    

2019-04-18 01:56:45 (80.6 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
# Process data set
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [0]:
from keras import models
from keras import layers

# Add Dense layers on top of model
model = models.Sequential()

vgg.trainable = False

model.add(vgg)

# Flatten output of VGG
model.add(layers.Flatten())

# FC layers and final classification layer
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))



In [7]:
from keras.preprocessing.image import ImageDataGenerator

#Create data augmentation generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')


validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')


# Compile model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
# enable fine tuning for last conv layer
vgg.trainable = True

for layer in vgg.layers:
  if layer.name == 'block5_conv3':
    layer.trainable = True
  else:
    layer.trainable = False

    
for layer in vgg.layers:
  print(layer.name, layer.trainable)
  
  
# Compile model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool False
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                262176    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
________________________________________________________

In [10]:
# Train model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
100/100 [==============================] - 764s 8s/step - loss: 0.6971 - acc: 0.6270 - val_loss: 0.7890 - val_acc: 0.6860
Epoch 2/20
100/100 [==============================] - 752s 8s/step - loss: 0.4383 - acc: 0.8080 - val_loss: 0.3093 - val_acc: 0.8700
Epoch 3/20
100/100 [==============================] - 758s 8s/step - loss: 0.3802 - acc: 0.8355 - val_loss: 0.3309 - val_acc: 0.8360
Epoch 4/20
100/100 [==============================] - 757s 8s/step - loss: 0.3380 - acc: 0.8560 - val_loss: 0.2847 - val_acc: 0.9120
Epoch 5/20
100/100 [==============================] - 755s 8s/step - loss: 0.3583 - acc: 0.8600 - val_loss: 0.2258 - val_acc: 0.9160
Epoch 6/20
100/100 [==============================] - 756s 8s/step - loss: 0.3412 - acc: 0.8720 - val_loss: 0.2138 - val_acc: 0.9240
Epoch 7/20
100/100 [==============================] - 754s 8s/step - loss: 0.2916 - acc: 0.8810 - val_loss: 0.2618 - val_acc: 0.9110
Epoch 8/20
100/100 [=

KeyboardInterrupt: ignored

In [0]:
#plot results
import matplotlib as mpl
import matplotlib.pyplot as plt

epoch = []
for i in range(20):
  epoch.append(i+1)
  
plt.plot(epoch, history.history['val_acc'], label='Validation accuracy')
plt.plot(epoch, history.history['acc'], label='Training accuracy')
plt.xlabel("# of epochs")
plt.ylabel("Accuracy")
plt.title("Results")
plt.legend(loc="upper left", fontsize=16)
plt.show()